# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.46,81,88,4.85,PN,1682782780
1,1,blackmans bay,-43.0167,147.3167,8.07,81,5,3.37,AU,1682782782
2,2,talnakh,69.4865,88.3972,-3.98,75,7,7.19,RU,1682782783
3,3,lebu,-37.6167,-73.6500,12.81,83,99,5.46,CL,1682782783
4,4,saint-pierre,-21.3393,55.4781,25.82,69,40,5.14,RE,1682782784


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# Configure the map
pmap_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map plot
pmap_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_condition = ideal_condition.dropna()

# Display sample data
ideal_condition.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
188,188,hilo,19.7297,-155.0900,22.93,88,0,2.06,US,1682782932
208,208,laguna,38.4210,-121.4238,25.05,70,0,2.57,US,1682782946
228,228,ixtapa,20.7000,-105.2000,26.24,57,0,2.57,MX,1682782962
240,240,andovoranto,-18.9500,49.1000,23.35,80,0,3.24,MG,1682782971
298,298,capitan bado,-23.2667,-55.5333,24.58,70,0,3.69,PY,1682783010


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
188,hilo,US,19.7297,-155.0900,88,
208,laguna,US,38.4210,-121.4238,70,
228,ixtapa,MX,20.7000,-105.2000,57,
240,andovoranto,MG,-18.9500,49.1000,80,
298,capitan bado,PY,-23.2667,-55.5333,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
import json
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = json.dumps(name_address)
    name_address = json.loads(name_address)
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
ixtapa - nearest hotel: Ma. Cristina
andovoranto - nearest hotel: Le nirvana
capitan bado - nearest hotel: No hotel found
les avirons - nearest hotel: Floralys
hood river - nearest hotel: Hood River Hotel
sucre - nearest hotel: Homestay Bertha Sucre
siwa oasis - nearest hotel: فندق الكيلانى
xanxere - nearest hotel: City Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
188,hilo,US,19.7297,-155.0900,88,Dolphin Bay Hotel
208,laguna,US,38.4210,-121.4238,70,Holiday Inn Express & Suites
228,ixtapa,MX,20.7000,-105.2000,57,Ma. Cristina
240,andovoranto,MG,-18.9500,49.1000,80,Le nirvana
298,capitan bado,PY,-23.2667,-55.5333,70,No hotel found
302,les avirons,RE,-21.2418,55.3394,78,Floralys
315,hood river,US,45.7054,-121.5215,61,Hood River Hotel
332,sucre,BO,-19.0333,-65.2627,36,Homestay Bertha Sucre
485,siwa oasis,EG,29.2041,25.5195,19,فندق الكيلانى
560,xanxere,BR,-26.8769,-52.4042,47,City Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="EsriImagery",
    frame_width=750,
    frame_height=550,
    size=10,
    color="City",
    hover_cols=["Lng","Lat","City", "Humidity", "Hotel Name", "Country"]
)

# Display the map plot
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)